In [3]:
import os 
import numpy as np 
import pandas as pd 

### handle data from Nansen NFT Paradise 
- I don't have the csv access, so I copy-pasted frm the website and rehandled the data 
- already filtered (floor price between 0.05 ~ 0.12)
- from https://pro.nansen.ai/nft-paradise


In [4]:
# load data 
data_path = os.path.join(os.getcwd(), "data")
file_name = os.path.join(data_path, "nansen_nft_paradise_1.xlsx")
sheet_to_df_map = pd.read_excel(file_name, sheet_name=None, header=None)

# reshape data 
col_names = ["NFT_collections", "Volumne_ETH", "Mkt_Cap_ETH", "Avg_Price_ETH", 
             "Floor_ETH", "Floor_24H", "Floor_3D", "#Wallets", "Deployed"]
lst_data = []
for sheet in sheet_to_df_map:
    df_new = pd.DataFrame(data = sheet_to_df_map[sheet].values.reshape(-1, 9), columns=col_names)
    lst_data.append(df_new)

# concat and save data 
df_data = pd.concat(lst_data)
df_data.reset_index(inplace=True, drop=True)
df_data = df_data.loc[df_data["NFT_collections"] != "NFT Collection"]
df_data.drop_duplicates(subset= "NFT_collections", inplace=True)
df_data.to_excel(os.path.join(data_path, "nansen_nft_paradise_handled.xlsx"))
df_data.head(), df_data.shape

(  NFT_collections Volumne_ETH Mkt_Cap_ETH Avg_Price_ETH Floor_ETH Floor_24H  \
 0     TrippyToadz         426         154        0.0735     0.068         0   
 1  Juicebox Frens         211         950        0.1363    0.0859      -0.2   
 2     Mini Supers         149         139        0.0728     0.084         -   
 3    MPL Official          75        2299        0.2299      0.11      -0.3   
 4       ToonSquad          65         487        0.0809    0.0558   -0.0084   
 
   Floor_3D #Wallets Deployed  
 0        -     2496  40h ago  
 1     0.24     2890   5d ago  
 2        -     1650  37h ago  
 3    -0.46     4033   6d ago  
 4     -0.3     2454   5d ago  ,
 (152, 9))

### Twitter hotness filtration 

cretiria (can be customized)
- mean_tweets_per_week
- mean_tweets_per_week_exclu_retweets
- mean_like_count
- mean_retweet_count
- mean_reply_count

In [5]:
import datetime, tqdm 
from twitter_giveaway_bot import *

# connect to twitter api v2 
client = connect()

In [6]:
# Twitter hotness filtration 
def process_project_from_tweeter(client, project_name):

    # print("-*80")
    # print(f"project_name: {project_name}")
    # init dataframe to store value 
    df_data = pd.DataFrame(columns=["account", "NFT_collections", "followers_count", "following_count", "tweet_count", 
                                    "mean_tweets_per_week", "mean_tweets_per_week_exclu_retweets", "mean_like_count", "mean_retweet_count", "mean_reply_count", "twitter_url"])
    df_data.set_index("account", inplace=True)

    # search related recent tweets 
    recent_tweets =  client.search_recent_tweets(project_name, user_auth=True, max_results=20)
    if recent_tweets.data is not None:
        # find all twitter accounts in the recent tweets  
        lst = [re.findall(r"@([a-zA-Z0-9_]+)", tweet["text"]) for tweet in recent_tweets.data]
        potential_twitter_accounts = list(set([item for sublist in lst for item in sublist]))
        # print(f"potential_twitter_accounts: {potential_twitter_accounts}")
        # find the true one 
        # assume that the twitter account should at least contains one part of the project name  
        lst2 = [account.lower() for account in potential_twitter_accounts]
        project_name_lower =  project_name.lower().split(" ")
        true_potential_twitter_accounts = set([potential_twitter_accounts[i] for i in range(len(lst2)) for j in project_name_lower if j in lst2[i]])
        # print(f"true_potential_twitter_accounts: {true_potential_twitter_accounts}")
        
        # if cant find the twitter accounts and the set is empty, we escape
        if true_potential_twitter_accounts:
            for account in true_potential_twitter_accounts:
                # get account info 
                user = client.get_user(username=account, user_fields=["public_metrics"], user_auth=True).data
                if user:
                    account_info = user["public_metrics"]
                    followers_count = account_info["followers_count"]
                    following_count = account_info["following_count"]
                    tweet_count = account_info["tweet_count"]
                    
                    # get account recent tweets 
                    latest_tweets = client.get_users_tweets(
                        user["id"],
                        user_auth=True,
                        max_results=100,
                        tweet_fields=["text", "public_metrics"],
                        start_time = datetime.datetime.now() - datetime.timedelta(days=14)
                    ).data

                    if latest_tweets:
                        # statistics 
                        # !! for average tweets per week, since we can only access 100 recent tweets, if the number of recent tweets
                        # is larger than 100, this measure is underestimated  
                        mean_tweets_per_week = len(latest_tweets) / 14 * 7
                        total_number_of_tweets_exclu_retweets = sum([1 if "RT" not in one_tweet["text"] else 0 for one_tweet in latest_tweets])
                        mean_tweets_per_week_exclu_retweets = total_number_of_tweets_exclu_retweets  / 14 * 7
                        mean_like_count = np.mean([one_tweet["public_metrics"]["like_count"] for one_tweet in latest_tweets])
                        mean_retweet_count = np.mean([one_tweet["public_metrics"]["retweet_count"] for one_tweet in latest_tweets])
                        mean_reply_count = np.mean([one_tweet["public_metrics"]["reply_count"] for one_tweet in latest_tweets])

                        # 
                        twitter_url = f"https://twitter.com/{account}"
                        df_data.loc[account, :] = [project_name, followers_count, following_count, tweet_count, 
                                                mean_tweets_per_week, mean_tweets_per_week_exclu_retweets, mean_like_count, mean_retweet_count, mean_reply_count,
                                                twitter_url]

    # else:
    #     print(f"{project_name} dont have recent tweets")
    
    return df_data


In [7]:
# test 
project_name = "TrippyToadz"
df = process_project_from_tweeter(client, project_name)
# df.to_excel(f"{project_name}.xlsx") 

In [8]:
lst_df_tweets_stats = []
for project_name in tqdm.tqdm(df_data["NFT_collections"][:]):
    try:
        df_tweets_stats = process_project_from_tweeter(client, project_name)
        lst_df_tweets_stats.append(df_tweets_stats)
    except Exception:
        # twitter has request limits for API 
        # after 15 minites 
        print("Need to relax for 16 minites")
        time.sleep(16 * 60 * 60)
    time.sleep(0.01)
df_tweets_stats_all = pd.concat(lst_df_tweets_stats)
df_tweets_stats_all.reset_index(inplace=True)
df_tweets_stats_all.drop_duplicates(subset="account", inplace=True)

100%|██████████| 152/152 [02:05<00:00,  1.21it/s]


In [9]:
# merge data and statistics 
df_results = pd.merge(df_data, df_tweets_stats_all, how="inner")
df_results.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_original.xlsx"))

df_results

,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,followers_count,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url
0,TrippyToadz,426,154,0.0735,0.068,0,-,2496,40h ago,trippytoadznft,23252,1214,982,50.0,35.0,123.43,114.01,33.18,https://twitter.com/trippytoadznft
1,Juicebox Frens,211,950,0.1363,0.0859,-0.2,0.24,2890,5d ago,Juiceboxfrens,6705,1479,1030,50.0,42.5,20.37,11.09,5.47,https://twitter.com/Juiceboxfrens
2,Juicebox Frens,211,950,0.1363,0.0859,-0.2,0.24,2890,5d ago,JuiceboxFrens,6705,1479,1030,50.0,42.5,20.37,11.09,5.47,https://twitter.com/JuiceboxFrens
3,Mini Supers,149,139,0.0728,0.084,-,-,1650,37h ago,minisupersnft,11574,2,199,50.0,26.0,18.17,27.19,2.02,https://twitter.com/minisupersnft
4,MPL Official,75,2299,0.2299,0.11,-0.3,-0.46,4033,6d ago,official_giftt,62,156,883,3.5,0.5,0.0,94.0,0.0,https://twitter.com/official_giftt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Seabums,1.62,222,0.038,0.03,0.0843,0.5,3198,109d ago,SeabumsNFTs,5246,430,429,26.5,16.5,9.792453,6.641509,2.207547,https://twitter.com/SeabumsNFTs
156,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,metaguardians,24227,242,1845,21.5,15.5,32.418605,20.023256,7.55814,https://twitter.com/metaguardians
157,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,TheCryptoLark,933911,455,40987,50.0,38.5,1148.93,150.18,246.97,https://twitter.com/TheCryptoLark
158,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,SkyGuardians_C,3387,88,690,45.0,3.0,0.266667,115.933333,0.044444,https://twitter.com/SkyGuardians_C


In [10]:
# filtration 
con_followers_count = df_results.loc[:, "followers_count"] > 40000
con_mean_tweets_per_week_exclu_retweets = df_results.loc[:, "mean_tweets_per_week_exclu_retweets"] > 2
con_mean_like_count = df_results.loc[:, "mean_like_count"] > 10
con_mean_retweet_count = df_results.loc[:, "mean_retweet_count"] > 10
con_mean_reply_count= df_results.loc[:, "mean_reply_count"] > 10
print(f"% con_followers_count: {np.mean(con_followers_count)}")
print(f"% con_mean_tweets_per_week_exclu_retweets: {np.mean(con_mean_tweets_per_week_exclu_retweets)}")
print(f"% con_mean_like_count: {np.mean(con_mean_like_count)}")
print(f"% con_mean_retweet_count: {np.mean(con_mean_retweet_count)}")
print(f"% con_mean_reply_count: {np.mean(con_mean_reply_count)}")

# combind condition 
con_all = con_followers_count & con_mean_tweets_per_week_exclu_retweets & con_mean_like_count & con_mean_retweet_count & con_mean_reply_count
df_results_filtered = df_results.loc[con_all, :]
print(f"total {df_results.count()[0]} NFT projects within floor price range [0.05 ~ 0.12]")
print(f"there are {df_results_filtered.count()[0]} satisfying the twitter condition")
df_results_filtered.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_filtered.xlsx"))
df_results_filtered

% con_followers_count: 0.2875
% con_mean_tweets_per_week_exclu_retweets: 0.91875
% con_mean_like_count: 0.66875
% con_mean_retweet_count: 0.69375
% con_mean_reply_count: 0.375
total 160 NFT projects within floor price range [0.05 ~ 0.12]
there are 31 satisfying the twitter condition


,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,followers_count,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url
8,Villagers of XOLO,50,3274,0.2669,0.11,-0.2,-0.44,4858,10d ago,PlanetXOLO,45628,2,51,13.0,10.5,239.0,76.0,14.730769,https://twitter.com/PlanetXOLO
13,Super Ordinary Villains - SOV,14,797,0.0897,0.07,-0.23,-0.41,4919,14d ago,SOVillains,77204,9,198,24.0,9.5,182.416667,181.0625,47.0,https://twitter.com/SOVillains
17,Squishiverse,11,1335,0.1502,0.1,-0.2,0.0903,1960,40d ago,SquishiverseNFT,63064,82,339,50.0,29.0,56.57,101.68,31.45,https://twitter.com/SquishiverseNFT
18,Rubber Duck Bath Party,8.47,974,0.0974,0.085,-0.25,-0.34,5929,24d ago,RubberDuckBP,60207,4,1094,22.5,18.5,198.688889,70.355556,25.466667,https://twitter.com/RubberDuckBP
19,Wabi Sabi Collective,8.42,673,0.1186,0.105,-0.13,0.28,1919,20d ago,WabiSabiNFT,113736,17,423,34.0,23.5,1295.308824,1280.397059,1058.176471,https://twitter.com/WabiSabiNFT
20,Crypto Bears,8.03,591,0.0887,0.079,1.03,0.58,3796,39d ago,cryptomanran,593380,2851,12899,50.0,42.5,292.84,41.12,74.79,https://twitter.com/cryptomanran
23,Crypto Bears,8.03,591,0.0887,0.079,1.03,0.58,3796,39d ago,crypto_bitlord7,197697,462,9314,50.0,44.5,120.27,458.56,27.79,https://twitter.com/crypto_bitlord7
26,Wulfz,7.98,1082,0.1376,0.093,0.45,0.35,991,85d ago,wulfznft,88134,13,454,28.5,17.5,763.245614,752.508772,719.614035,https://twitter.com/wulfznft
32,The Sunnies,7.24,720,0.0787,0.066,-0.14,-0.27,4678,24d ago,thesunniesnft,41600,27,380,29.0,15.5,115.137931,122.931034,13.344828,https://twitter.com/thesunniesnft
42,The GOAT Society Season 2,4.9,79,0.084,0.065,-0.54,-0.35,359,4d ago,TheNFTFocal,58494,9,4816,34.0,25.5,134.441176,12.279412,194.117647,https://twitter.com/TheNFTFocal


### Find discord urls 



In [11]:
import re
import requests
from bs4 import BeautifulSoup

# send request and return soup
def quickSoup(url):
    try:
        header = {}
        header['User-Agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        soup = BeautifulSoup(requests.get(url, headers=header, timeout=10).content, 'html.parser')
        return soup
    except Exception:
        return None
    
def find_discord_url(username):
   
    print("-"*80)
    print(f"username: {username}")
    discord_url = None
    
    ##
    # find discord url from twitter
    # 1. discription
    # 2. linktree 
    user = client.get_user(username=username, user_fields=[ "entities"]).data
    entities = user["entities"]
    for i in entities:
        if "urls" in entities[i]:
            urls = entities[i]["urls"]
            for j in urls:
                expanded_url = j["expanded_url"]
                if any(w in expanded_url for w in ["Discord", "discord"]):
                    discord_url = expanded_url
                if "linktr" in expanded_url:
                    # print("through linktr")
                    # print(expanded_url)
                    # request the linktr and find the discord url
                    soup = quickSoup(expanded_url)
                    for a in soup.find_all('a', href=True):
                        if any(w in a["href"] for w in ["Discord", "discord"]):
                            # print("Found the discord URL:", a['href'])
                            discord_url = a['href']
    
    return discord_url
                
    
def find_number_of_memeber_in_discord(discord_url):
        # scrape discord info 
        soup = quickSoup(discord_url)
        content = ''.join([str(i) for i in soup.contents])
        phrase = '_'.join(re.findall(r"[0-9A-Za-z, ]+ member[s].", content))
        n_members = re.findall(r"[0-9,]+", phrase)[0].replace(",","").replace(" member", "")
        print(n_members)
    
        return n_members



In [16]:
df_results_filtered.loc[:, "n_discord"] = 0
df_results_filtered.loc[:, "url_d"] = ""
# get number of memebers in discord:
for i in range(len(df_results_filtered.index)):
    discord_url = find_discord_url(df_results_filtered.account.iloc[i])
    print(f"discord_url: {discord_url}")
    if discord_url is not None:
        n_members = find_number_of_memeber_in_discord(discord_url)
        df_results_filtered.loc[df_results_filtered.index[i], "n_discord"] = n_members
        df_results_filtered.loc[df_results_filtered.index[i], "url_d"] = str(discord_url)
        # df_results_filtered["n_discord"].iloc[i] = n_members
        # df_results_filtered["url_d"].iloc[i] = str(discord_url)


--------------------------------------------------------------------------------
username: PlanetXOLO
discord_url: http://Discord.gg/PlanetXOLO
13958
--------------------------------------------------------------------------------
username: SOVillains
discord_url: https://discord.gg/sovillains
21502
--------------------------------------------------------------------------------
username: SquishiverseNFT
discord_url: https://discord.gg/squishiverse
12304
--------------------------------------------------------------------------------
username: RubberDuckBP
discord_url: None
--------------------------------------------------------------------------------
username: WabiSabiNFT
discord_url: http://discord.gg/wabisabicollective
22229
--------------------------------------------------------------------------------
username: cryptomanran
discord_url: None
--------------------------------------------------------------------------------
username: crypto_bitlord7
discord_url: None
-------------

In [17]:
df_results_filtered = df_results_filtered.astype({"n_discord":float})
df_results_filtered2 = df_results_filtered.loc[df_results_filtered["n_discord"]> 15000, :]
df_results_filtered2.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_filtered.xlsx"))
df_results_filtered2

,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,...,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url,n_discord,url_d
13,Super Ordinary Villains - SOV,14,797,0.0897,0.07,-0.23,-0.41,4919,14d ago,SOVillains,...,9,198,24.0,9.5,182.416667,181.0625,47.0,https://twitter.com/SOVillains,21502.0,https://discord.gg/sovillains
19,Wabi Sabi Collective,8.42,673,0.1186,0.105,-0.13,0.28,1919,20d ago,WabiSabiNFT,...,17,423,34.0,23.5,1295.308824,1280.397059,1058.176471,https://twitter.com/WabiSabiNFT,22229.0,http://discord.gg/wabisabicollective
26,Wulfz,7.98,1082,0.1376,0.093,0.45,0.35,991,85d ago,wulfznft,...,13,454,28.5,17.5,763.245614,752.508772,719.614035,https://twitter.com/wulfznft,21972.0,http://discord.gg/wulfz
44,C86 Cyborg,4.85,589,0.0685,0.074,1.2,0.87,3902,78d ago,Cyborg86nft,...,20,235,23.0,17.5,47.152174,50.565217,12.521739,https://twitter.com/Cyborg86nft,301726.0,http://discord.com/invite/cyborg86
46,FishyFam,4.38,1326,0.1326,0.118,-0.1,-0.12,5966,58d ago,fishyfamNFT,...,525,435,19.5,15.5,486.871795,296.179487,224.025641,https://twitter.com/fishyfamNFT,74621.0,http://discord.gg/fishyfam
49,Crypto Hippos NFT,4.18,541,0.0755,0.048,-0.0303,-0.0233,2429,29d ago,CryptoHipposNFT,...,20,211,17.0,8.0,139.529412,129.088235,23.852941,https://twitter.com/CryptoHipposNFT,22774.0,https://discord.gg/cryptohippos
54,CerealClub,4.11,1277,0.1283,0.0899,-0.13,-0.23,4110,36d ago,cerealclubnft,...,21,334,34.0,15.5,216.735294,212.647059,168.323529,https://twitter.com/cerealclubnft,31818.0,http://discord.gg/cerealclub
63,House Of Legends,3.59,1995,0.1996,0.0599,0.0328,0.2,2673,93d ago,veve_official,...,196,4398,46.0,43.5,409.413043,74.021739,64.521739,https://twitter.com/veve_official,116272.0,https://discord.gg/veve
87,Gooniez Gang,2.57,1200,0.135,0.085,-0.0957,-0.12,5734,41d ago,GooniezNFT,...,123,568,15.5,11.5,204.451613,187.774194,114.806452,https://twitter.com/GooniezNFT,134313.0,https://discord.gg/gooniez
96,The Ramen Shop NFT,2.25,25,0.0487,0.05,0.24,-0.11,284,5d ago,SoulZ_NFT,...,10,361,10.5,7.5,789.190476,984.809524,712.285714,https://twitter.com/SoulZ_NFT,96535.0,https://discord.com/invite/soulz
